# Load Data

In [ ]:
import tensorflow as tf

# Load the Fashion MNIST dataset
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize the data
x_train, x_test = x_train / 255.0, x_test / 255.0

### Data Prep

In [ ]:
# Subset your data the same 3 digit classes you used in the Random Forest problem from the last assignment.

# Again, You can totally copy any code over from the fmnist_examples notebook and modify it.
# You can use chatGPT, copilot, google, or other AI or online resources.
# Use each other, the goal is to complete the objectives and maybe learn something new, not to struggle to make up code on your own.
# We have prior notebooks, chatGPT, the internet, and each other for the rapids projects, so use them here if they will help!

# Extract the 3 classes from the training data

train_filter = (y_train == 5) | (y_train == 6)
x_train_nn = x_train[train_filter]
y_train_nn = y_train[train_filter]

# Extract the 3 classes from the test data

test_filter = (y_test == 5) | (y_test == 6)
x_test_nn = x_test[test_filter]
y_test_nn = y_test[test_filter]

print(y_train_nn)
y_train_nn_bin = (y_train_nn == 5).astype(int)
y_test_nn_bin = (y_test_nn == 5).astype(int)
print(y_train_nn_bin)

### Plot Data

In [ ]:
## Plot 6 to 10 images from your data

import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))

for i in range(8):
    plt.subplot(2, 4, i + 1)
    plt.imshow(x_train_nn[i], cmap='gray')
    plt.title(f'Label: {y_train_nn_bin[i]}')
    plt.axis('off')

plt.show()


# CNN with Pytorth

### Design the CNN Model

In [ ]:
# Define a CNN model. 
# You are welcome to use the model from the fmnist_examples notebook, or you can try to create a better one.
# Either way, I recommend starting with the model from the fmnist_examples notebook and modifying it as you like.

# Important: Remember that our output has 3 classes, not 2 like in the Fashion MNIST dataset,
# So you will have to change at least one thing in the model to accommodate this.

import torch
from torch.utils.data import DataLoader, TensorDataset

import torch.nn as nn
import torch.optim as optim

# Define the CNN model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.fc1 = nn.Linear(64 * 5 * 5, 128)
        self.fc2 = nn.Linear(128, 2)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 5 * 5)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

### Data Prep

In [ ]:
# Convert your data to PyTorch tensors and create DataLoader objects for training and testing

# Convert the data to PyTorch tensors
x_train_tensor = torch.tensor(x_train_nn.reshape(-1, 1, 28, 28), dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_nn_bin, dtype=torch.long)
x_test_tensor = torch.tensor(x_test_nn.reshape(-1, 1, 28, 28), dtype=torch.float32)
y_test_tensor = torch.tensor(y_test_nn_bin, dtype=torch.long)

# Create DataLoader for training and testing
train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
test_dataset = TensorDataset(x_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

### Fit the CNN

In [ ]:
# Initialize the model, loss function, and optimizer

t_model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(t_model.parameters(), lr=0.001)

# Train the model
# More epochs takes more time, but also helps your model to be more accurate.
# Report some notion of accuracy, loss or both as it builds to see how well your model is doing.

num_epochs = 10
for epoch in range(num_epochs):
    t_model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = t_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    # Calculate validation loss and accuracy
    t_model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for val_inputs, val_labels in test_loader:
            val_outputs = t_model(val_inputs)
            val_loss += criterion(val_outputs, val_labels).item()
            _, predicted = torch.max(val_outputs, 1)
            total += val_labels.size(0)
            correct += (predicted == val_labels).sum().item()
    
    # Compute metrics
    avg_train_loss = running_loss / len(train_loader)
    avg_val_loss = val_loss / len(test_loader)
    val_accuracy = correct / total
    
    # Print metrics rounded to 4 decimals
    print(f'Epoch {epoch + 1}/{num_epochs}, '
          f'Train Loss: {avg_train_loss:.4f}, '
          f'Validation Loss: {avg_val_loss:.4f}, '
          f'Validation Accuracy: {val_accuracy:.4f}')


# How does the test accuracy compare to that of the Random Forest model previously used?



### Plot Confusion Matrix

In [ ]:
# Compute and plot a confusion matrix.

'Your code here'

# Where are misclassifications occurring?
# How does your test accuracy compare to that of the Random Forest model previously made?

### Plot Misclassifications

In [ ]:
# Plot some of the misclassifications.

'Your code here'

# Can you see why the model made the misclassifications it did?

# CNN with Transformations

### Data Transformations

In [ ]:
# Implement some kind of Appropriate Data Transformation(s) for this data set.
# You can/should copy the train_mod function from the fmnist_examples_ml2 notebook, but you will have to 
# think about whether all of the transformations are appropriate for this data set and maybe adjust them.

# There is 1 specific modification you must make to the train_mod function! Other modifications are great, but optional.

from torchvision.datasets import MNIST
from torchvision import transforms

# Define data augmentation transformations
train_mod = transforms.Compose([
    "Your code here",
    transforms.ToTensor(),                     # Convert to tensor
    transforms.Normalize((0.5,), (0.5,))    # Normalize to [-1, 1]
])

test_mod = transforms.Compose([
    transforms.ToTensor(),                     # Convert to tensor
    transforms.Normalize((0.5,), (0.5,))       # Normalize to [-1, 1]
])

# Load the MNIST train dataset
train_dataset = MNIST(root="./data", train=True, transform=train_mod, download=True)

# Filter the dataset to only include the 3 classes of interest
# Convert to labels to 0, 1, 2
# Convert the dataset to a DataLoader object

"Your code here"

# Repeat for the loading, filtering, and conversions on the test dataset
test_dataset = MNIST(root="./data", train=False, transform=test_mod, download=True)

'Your code here'

# Plot some of the images in your train_loader. Check that your transformations provide
# a reasonable augmentation of the data. If they don't, adjust them in the train_mod function.

'Your code here'

### Fit the CNN

In [ ]:
# Run your CNN using the data loaders containing different transformations.
# This very well may not be better than the model trained on the original data, 
# since we don't expect our test data to be transformed. But try it out.

# Provide initializations for a new model and train the model.
# Report some notion of accuracy, loss or both as it builds to see how well your model is doing.

'Your code here'

# How does the test accuracy compare to other models you've built?